In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/MRNet


In [3]:
import os
import numpy as np
import glob
import csv
import keras
import scipy
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
from keras.models import load_model
from numpy.random import seed

from inception_model_generator import MRNet_inc_model,MRNet_inception_layer

import tensorflow as tf


Using TensorFlow backend.


In [0]:
label_type = 'ACL'
exam_view = 'sagittal'
model, cbs = MRNet_inc_model(1,0.00001,combination=[label_type, exam_view])
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['train'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['train'][exam_view])
neg = total - pos

class_weight = {0: 1, 1: neg/pos}
if neg < pos:
  class_weight = {0: pos/neg, 1: 1}

print(class_weight)
batch_size = 1
train_data_gen = MRNet_data_generator(data_path, p, l ,{0:1,1:1}, batch_size=1,model="inception",data_type='train', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)
test_data_gen = MRNet_data_generator(data_path, p, l,{0:1,1:1} ,batch_size=1,model="inception",data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)


model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception/"+label_type+"/"+exam_view+"/weights.37.hdf5")
model.fit(x=train_data_gen, validation_data = test_data_gen,epochs=50, initial_epoch = 37,verbose=True, class_weight=class_weight,callbacks=[cbs[0]],shuffle=False, use_multiprocessing=True, workers=6)

no dropout
no dropout
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mr_net_inception_layer_1 (MR (None, 1)                 21804833  
Total params: 21,804,833
Trainable params: 21,770,401
Non-trainable params: 34,432
_________________________________________________________________
{0: 1, 1: 4.16243654822335}
Initializing Data Generator:
factor :  1.0
model:  inception
data type:  train
Combination:  ACL  and  sagittal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/train
Number of inputs:  1017
input size:  (299, 299)
Initializing Data Generator:
factor :  1.0
model:  inception
data type:  test
Combination:  ACL  and  sagittal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/test
Number of inputs:  113
input size:  (299, 299)
Epoch 38/50
1017/1017 [==============================] - 577s 567ms/step - loss: 0.4737 - tp: 91

# **testing on validation dataset**

In [5]:

label_type = 'ACL'
exam_view = 'sagittal'

model, cbs = MRNet_inc_model(1,0.00001,combination=[label_type, exam_view])

# print(model(tf.ones((12, 24, 224, 224, 3))).shape)

model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception/"+label_type+"/"+exam_view+"/weights.45.hdf5")


data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

batch_size = 1
print(model.metrics_names)
valid_data_gen = MRNet_data_generator(data_path, p, l,{0:1,1:1} , batch_size=1,model="inception",data_type='valid', shuffle=False,label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)

d = model.evaluate_generator(valid_data_gen, 120, verbose=1)
print(d)
# model.predict_generator(valid_data_gen, 120 ,verbose=1)

no dropout
no dropout
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mr_net_inception_layer_12 (M (None, 1)                 21804833  
Total params: 21,804,833
Trainable params: 21,770,401
Non-trainable params: 34,432
_________________________________________________________________
['loss', 'tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall', 'auc']
Initializing Data Generator:
factor :  1.0
model:  inception
data type:  valid
Combination:  ACL  and  sagittal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/valid
Number of inputs:  120
input size:  (299, 299)
120/120 [==============================] - 18s 148ms/step
[1.6406103372573853, 17.491666793823242, 4.808333396911621, 29.350000381469727, 8.850000381469727, 0.7555496096611023, 0.7504664659500122, 0.678683876991272, 0.7401111721992493]
